In [1]:
import json
import fitz  # PyMuPDF
import pandas as pd
import re
import os
from pathlib import Path
from typing import Dict, List, Tuple
from datetime import datetime

# LangChain for chunking & embeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings

# Paths
PDF_DIR = Path("data/pdfs")
CHUNKS_DIR = Path("data/chunks")
METADATA_PATH = Path("metadata/canonical/fino_policies_metadata.json")
FAQ_PATH = Path("data/raw/fino_full_qa_dump.txt")  

# Create directories
PDF_DIR.mkdir(exist_ok=True)
CHUNKS_DIR.mkdir(exist_ok=True)

print("✅ Imports loaded")
print(f"📁 PDF_DIR: {PDF_DIR}")
print(f"📁 CHUNKS_DIR: {CHUNKS_DIR}")
print(f"📄 METADATA: {METADATA_PATH.exists()}")
print(f"📄 FAQ: {FAQ_PATH.exists()}")


c:\Users\risha\anaconda3\envs\fino\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports loaded
📁 PDF_DIR: data\pdfs
📁 CHUNKS_DIR: data\chunks
📄 METADATA: True
📄 FAQ: True


In [2]:
# Load canonical metadata
with open(METADATA_PATH, 'r', encoding='utf-8') as f:
    policies = json.load(f)

print(f"📋 Loaded {len(policies)} policy versions")
print(f"🎯 Active policies: {sum(1 for p in policies if p['status'] == 'Active')}")
print(f"📂 PDFs in data/pdfs/: {len(list(PDF_DIR.glob('*.pdf')))}")

# Validate all PDF paths exist
missing_pdfs = []
for policy in policies:
    pdf_path = PDF_DIR / policy['file_name']
    if not pdf_path.exists():
        missing_pdfs.append(policy['file_name'])

if missing_pdfs:
    print(f"Missing {len(missing_pdfs)} PDFs:")
    for pdf in missing_pdfs[:5]:
        print(f"   - {pdf}")
else:
    print("✅ All PDFs found")

print("\n" + "="*50)


📋 Loaded 16 policy versions
🎯 Active policies: 14
📂 PDFs in data/pdfs/: 16
✅ All PDFs found



In [ ]:
def universal_extract(pdf_path: Path) -> Tuple[str, Dict]:
    """
    ROBUST EXTRACTION: Layer 1-4 (Blocks + Tables + Fallback + Cleaning)
    Returns (full_text, extraction_stats)
    """
    doc = fitz.open(pdf_path)
    full_text_parts = []
    extraction_stats = {
        "total_pages": len(doc),
        "text_blocks": 0,
        "table_blocks": 0,
        "raw_chars": 0
    }
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        
        # LAYER 1: BLOCKS (text + table structure)
        blocks = page.get_text("blocks")
        page_content = []
        
        for block in blocks:
            x0, y0, x1, y1, block_text, _, block_type = block
            if block_text.strip():
                if block_type == 0:  # Text block
                    page_content.append(block_text)
                    extraction_stats["text_blocks"] += 1
                else:  # Table/graphics
                    rect = fitz.Rect(x0, y0, x1, y1)
                    table_text = page.get_text("text", clip=rect)
                    if table_text.strip():
                        page_content.append(f"\n[TABLE]{table_text.strip()}[/TABLE]\n")
                        extraction_stats["table_blocks"] += 1
        
        # LAYER 2: TABLES (PyMuPDF table detection)
        tables = page.find_tables()
        for table in tables:
            try:
                df = pd.DataFrame(table.extract())
                table_str = df.to_string(index=False, header=False)
                page_content.append(f"\n[TABLE-DATA]\n{table_str}\n[/TABLE-DATA]")
                extraction_stats["table_blocks"] += 1
            except:
                pass
        
        # LAYER 3: RAW FALLBACK
        raw_text = page.get_text()
        if raw_text.strip() and len(raw_text) > 50:
            page_content.append(raw_text)
            extraction_stats["raw_chars"] += len(raw_text)
        
        # LAYER 4: Aggressive cleaning
        page_text = "\n\n".join(page_content)
        page_text = re.sub(r'Classification: Internal.*?(?=\n{2,}|\Z)', '', page_text, flags=re.IGNORECASE | re.DOTALL)
        page_text = re.sub(r'FINO Payments Bank Limited\s*\n?', '', page_text, flags=re.IGNORECASE)
        page_text = re.sub(r'^\s*\d+\s*\n?', '', page_text, flags=re.MULTILINE)
        page_text = re.sub(r'\n{5,}', '\n\n\n', page_text)
        page_text = re.sub(r'\s{4,}', ' ', page_text)
        
        if len(page_text.strip()) > 30:
            full_text_parts.append(page_text.strip())
    
    doc.close()
    
    full_text = "\n\n=== PAGE BREAK ===\n\n".join(full_text_parts)
    extraction_stats["total_chars"] = len(full_text)
    extraction_stats["total_words"] = len(full_text.split())
    
    return full_text, extraction_stats

print("✅ Universal Extractor loaded (Layers 1-4)")


✅ Universal Extractor loaded (Layers 1-4)


In [4]:
# # BGE-M3 specs
# BGE_M3_MAX_TOKENS = 8192  # Context window
# BGE_M3_MAX_CHUNK_CHARS = 4000  # Safe embedding limit (~75% of max)
# CHUNK_OVERLAP_CHARS = 200  # Semantic overlap

# print(f"🔧 BGE-M3 Config:")
# print(f"   Max tokens: {BGE_M3_MAX_TOKENS:,}")
# print(f"   Max chunk chars: {BGE_M3_MAX_CHUNK_CHARS}")
# print(f"   Overlap: {CHUNK_OVERLAP_CHARS} chars")

# # Initialize embeddings for semantic chunking
# embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
# print("✅ BGE-M3 embeddings initialized")


In [7]:
E5_MAX_TOKENS = 1024          # Effective context window for E5
E5_MAX_CHUNK_CHARS = 2000     # Safe chunk size for embeddings
CHUNK_OVERLAP_CHARS = 200     # Semantic overlap 

print(f"🔧 E5-Large Config:")
print(f"   Max tokens: {E5_MAX_TOKENS:,}")
print(f"   Max chunk chars: {E5_MAX_CHUNK_CHARS}")
print(f"   Overlap: {CHUNK_OVERLAP_CHARS} chars")

# Initialize embeddings for semantic chunking 
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="D:/models/e5-large"   # local multilingual-e5-large
)

print("✅ E5-Large embeddings initialized")


🔧 E5-Large Config:
   Max tokens: 1,024
   Max chunk chars: 2000
   Overlap: 200 chars


No sentence-transformers model found with name D:/models/e5-large. Creating a new one with mean pooling.


✅ E5-Large embeddings initialized


In [ ]:
# Process ALL Active policies 
active_policies = [p for p in policies if p["status"] == "Active"]
print(f"🚀 Processing {len(active_policies)} ACTIVE policies...\n")

all_chunks = []
extraction_summary = []

for idx, policy in enumerate(active_policies, 1):
    pdf_path = PDF_DIR / policy['file_name']
    
    # FIXED: Safe policy_name access
    policy_name = policy.get('policy_name', policy.get('file_name', 'Unknown Policy'))[:50]
    
    print(f"[{idx}/{len(active_policies)}] 📄 {policy['file_name']}")
    print(f"   Policy: {policy_name}...")
    
    # Universal extraction
    full_text, stats = universal_extract(pdf_path)
    print(f"   📊 Extraction: {stats['total_chars']:,} chars, {stats['total_words']:,} words")
    print(f"   📊 Blocks: {stats['text_blocks']} text + {stats['table_blocks']} tables")
    
    extraction_summary.append({
        **policy,
        "policy_name_display": policy_name,  # Add fallback
        "extraction_stats": stats
    })
    
    # Semantic chunking with E5-Large (your config)
    chunker = SemanticChunker(
        embeddings,
        breakpoint_threshold_type="percentile",
        breakpoint_threshold_amount=85
    )
    
    raw_chunks = chunker.split_text(full_text)
    
    # Filter + validate chunks (E5 limits)
    valid_chunks = []
    for i, chunk in enumerate(raw_chunks):
        chunk_len = len(chunk)
        if 150 <= chunk_len <= E5_MAX_CHUNK_CHARS:  # Your E5 config
            valid_chunks.append({
                "policy_id": policy["policy_id"],
                "version_number": policy["version_number"],
                "chunk_id": f"{policy['policy_id']}_{policy['version_number']}_{i:04d}",
                "chunk_index": i,
                "text": chunk,
                "text_length": chunk_len,
                "word_count": len(chunk.split()),
                "metadata": policy,
                "extraction_stats": stats
            })
    
    all_chunks.extend(valid_chunks)
    
    # Save per-policy chunks
    policy_chunks_file = CHUNKS_DIR / f"{policy['policy_id']}_{policy['version_number']}.json"
    with open(policy_chunks_file, 'w', encoding='utf-8') as f:
        json.dump(valid_chunks, f, indent=2, ensure_ascii=False)
    
    print(f"   ✂️  {len(valid_chunks)}/{len(raw_chunks)} valid chunks")
    print(f"   📁 Saved: {policy_chunks_file.name}")
    print(f"   📏 Sizes: {min(c['text_length'] for c in valid_chunks) if valid_chunks else 0}-"
          f"{max(c['text_length'] for c in valid_chunks) if valid_chunks else 0} chars")
    print("-" * 80)
    
    # Sample first chunk
    if valid_chunks:
        print(f"   🧪 Sample: {valid_chunks[0]['text'][:150]}...")
    print()

# Global summary
print("=" * 80)
print("🎉 EXTRACTION + CHUNKING COMPLETE!")
print(f"📊 Total chunks: {len(all_chunks):,}")
print(f"📝 Total chars: {sum(c['text_length'] for c in all_chunks):,}")
print(f"📁 Chunk files: {len(list(CHUNKS_DIR.glob('*.json')))}")

# Save extraction report
with open("data/extraction_report.json", 'w', encoding='utf-8') as f:
    json.dump(extraction_summary, f, indent=2, ensure_ascii=False)

print(f"✅ Extraction report: data/extraction_report.json")
print(f"✅ All chunks ready for embeddings → data/chunks/")
print(f"\n🎯 Next: Cell 5 (Chunk validation + embeddings)")


🚀 Processing 14 ACTIVE policies...

[1/14] 📄 Comprehensive Deposit Policy - V7 (1).pdf
   Policy: Comprehensive Deposit Policy - V7 (1).pdf...
Consider using the pymupdf_layout package for a greatly improved page layout analysis.
   📊 Extraction: 76,348 chars, 11,270 words
   📊 Blocks: 224 text + 5 tables
   ✂️  68/88 valid chunks
   📁 Saved: comprehensivedepositpolicy_7.json
   📏 Sizes: 154-1982 chars
--------------------------------------------------------------------------------
   🧪 Sample: Classification: Public Comprehensive Deposit Policy V7.0 
Page 1 Comprehensive Deposit Policy Version 7.0 The policy as enclosed was last approved by ...

[2/14] 📄 1736412697_1362025_BSBDAPolicy.pdf
   Policy: 1736412697_1362025_BSBDAPolicy.pdf...
   📊 Extraction: 33,523 chars, 5,138 words
   📊 Blocks: 185 text + 6 tables
   ✂️  29/40 valid chunks
   📁 Saved: bsbdapolicy_1.json
   📏 Sizes: 206-1485 chars
--------------------------------------------------------------------------------
   🧪 Sample

In [13]:
# FAQ Processing (MERGE WITH PDF CHUNKS)
print("📄 Processing FAQ file...")

if FAQ_PATH.exists():
    with open(FAQ_PATH, 'r', encoding='utf-8') as f:
        faq_text = f.read()
    
    print(f"📄 FAQ raw: {len(faq_text):,} chars, {len(faq_text.split()):,} words")
    
    # FAQ metadata (simple)
    faq_metadata = {
        "policy_id": "faq_section",
        "version_number": "1.0",
        "status": "Active",
        "effective_date_ts": int(datetime.now().timestamp()),
        "expiry_date_ts": 4102425000,  # 2100
        "effective_date": datetime.now().strftime("%Y-%m-%d"),
        "expiry_date": "2100-01-01",
        "language": "en",
        "file_name": FAQ_PATH.name,
        "policy_name": "Fino Bank FAQs (Website)",
        "sr": "FAQ-001"
    }
    
    # Semantic chunk FAQ
    faq_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
    faq_raw_chunks = faq_chunker.split_text(faq_text)
    
    faq_chunks = []
    for i, chunk in enumerate(faq_raw_chunks):
        chunk_len = len(chunk)
        if 150 <= chunk_len <= 4000:
            faq_chunks.append({
                "policy_id": "faq_section",
                "version_number": "1.0",
                "chunk_id": f"faq_section_1.0_{i:04d}",
                "chunk_index": i,
                "text": chunk,
                "text_length": chunk_len,
                "word_count": len(chunk.split()),
                "metadata": faq_metadata,
                "extraction_stats": {
                    "total_chars": len(faq_text),
                    "total_words": len(faq_text.split()),
                    "source": "faq_txt"
                }
            })
    
    # Save FAQ chunks
    faq_output = CHUNKS_DIR / "faq_section_1.0.json"
    with open(faq_output, 'w', encoding='utf-8') as f:
        json.dump(faq_chunks, f, indent=2, ensure_ascii=False)
    
    all_chunks.extend(faq_chunks)
    print(f"✂️ FAQ: {len(faq_chunks)} chunks → {faq_output.name}")
    
    extraction_summary.append({
        **faq_metadata,
        "extraction_stats": {
            "total_chars": len(faq_text),
            "total_words": len(faq_text.split()),
            "source": "faq_txt"
        }
    })
else:
    print("⚠️ FAQ file not found: data/raw/faqs.txt")

print(f"\n📊 TOTAL (PDFs + FAQ): {len(all_chunks):,} chunks")


📄 Processing FAQ file...
📄 FAQ raw: 88,607 chars, 13,783 words
✂️ FAQ: 35 chunks → faq_section_1.0.json

📊 TOTAL (PDFs + FAQ): 834 chunks


In [8]:
# Load & Validate All Chunks (E5-Large)
import uuid
from datetime import datetime

# Verify chunking results first
chunk_files = list(CHUNKS_DIR.glob("*.json"))
print(f"📁 Found {len(chunk_files)} chunk files")
print("Sample chunk files:")
for f in chunk_files[:5]:
    with open(f, 'r', encoding='utf-8') as ff:
        chunks = json.load(ff)
    print(f"   {f.name}: {len(chunks)} chunks")

# Load ALL chunks (PDFs + FAQ)
all_policy_chunks = []
for chunk_file in sorted(chunk_files):
    with open(chunk_file, 'r', encoding='utf-8') as f:
        policy_chunks = json.load(f)
    all_policy_chunks.extend(policy_chunks)
    print(f"✅ Loaded {len(policy_chunks):,} chunks from {chunk_file.name}")

print(f"\n📊 TOTAL: {len(all_policy_chunks):,} chunks ready for E5 embedding")
print(f"📏 Chunk sizes: {min(c['text_length'] for c in all_policy_chunks)} - "
      f"{max(c['text_length'] for c in all_policy_chunks)} chars")

# Chunk quality check (E5-safe)
chunk_stats = {
    "total_chunks": len(all_policy_chunks),
    "avg_length": sum(c['text_length'] for c in all_policy_chunks) / len(all_policy_chunks),
    "policies_covered": len(set(c['policy_id'] for c in all_policy_chunks)),
    "avg_words": sum(c['word_count'] for c in all_policy_chunks) / len(all_policy_chunks),
    "max_chunk_chars": max(c['text_length'] for c in all_policy_chunks),
    "min_chunk_chars": min(c['text_length'] for c in all_policy_chunks)
}

print(f"\n📈 Quality stats (E5-Large optimized):")
for k, v in chunk_stats.items():
    if isinstance(v, float):
        print(f"   {k}: {v:.1f}")
    else:
        print(f"   {k}: {v}")

# E5 constraint check
unsafe_chunks = [c for c in all_policy_chunks if c['text_length'] > E5_MAX_CHUNK_CHARS]
if unsafe_chunks:
    print(f"\n⚠️  {len(unsafe_chunks)} chunks exceed E5 limit ({E5_MAX_CHUNK_CHARS} chars)")
else:
    print(f"\n✅ All chunks safe for E5 embedding (max {E5_MAX_CHUNK_CHARS} chars)")

print("\n✅ Chunks validated → Ready for E5-Large embeddings")


📁 Found 15 chunk files
Sample chunk files:
   attached_onappointmentofstatutoryauditors_1.json: 10 chunks
   bsbdapolicy_1.json: 29 chunks
   chequecollectionpolicy_1.json: 14 chunks
   citizenscharter_1.json: 27 chunks
   codeofbankscommittmenttocustomers_1.json: 44 chunks
✅ Loaded 10 chunks from attached_onappointmentofstatutoryauditors_1.json
✅ Loaded 29 chunks from bsbdapolicy_1.json
✅ Loaded 14 chunks from chequecollectionpolicy_1.json
✅ Loaded 27 chunks from citizenscharter_1.json
✅ Loaded 44 chunks from codeofbankscommittmenttocustomers_1.json
✅ Loaded 68 chunks from comprehensivedepositpolicy_7.json
✅ Loaded 36 chunks from customercompensationpolicy_1.json
✅ Loaded 23 chunks from customergrievancepolicy_1.json
✅ Loaded 179 chunks from customerservicepolicy_1.json
✅ Loaded 29 chunks from deceasedclaimsettlementpolicy_1.json
✅ Loaded 35 chunks from faq_section_1.0.json
✅ Loaded 207 chunks from kycamlcftpolicy_1.json
✅ Loaded 23 chunks from policyoncustomersuitabilityandappropriat

In [9]:
# Generate E5-Large Dense Embeddings
import numpy as np
from langchain_huggingface import HuggingFaceEmbeddings

print("🔄 Loading E5-Large embeddings (multilingual, local)...")

# E5-Large already loaded from Cell 3
# Verify it's ready
e5_embeddings = HuggingFaceEmbeddings(
    model_name="D:/models/e5-large",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# Test embedding on first chunk
test_chunk = all_policy_chunks[0]['text'][:500]
test_embedding = e5_embeddings.embed_query(test_chunk)
print(f"✅ E5-Large test embedding: {len(test_embedding)} dimensions")
print(f"   Model: multilingual-e5-large (local)")
print(f"   Embedding type: Dense (cosine similarity)")
print(f"   Normalization: L2 normalized")

# Generate embeddings for ALL chunks (BATCHED for efficiency)
print("\n🚀 Generating E5-Large embeddings for all chunks...")

all_embeddings_data = []
batch_size = 32  # E5 batch size for CPU
total_batches = (len(all_policy_chunks) + batch_size - 1) // batch_size

for batch_idx in range(0, len(all_policy_chunks), batch_size):
    batch_end = min(batch_idx + batch_size, len(all_policy_chunks))
    batch_chunks = all_policy_chunks[batch_idx:batch_end]
    
    # Extract texts for batch
    batch_texts = [c['text'] for c in batch_chunks]
    
    # E5 embedding (dense only - no sparse for local model)
    batch_embeddings = e5_embeddings.embed_documents(batch_texts)
    
    # Attach embeddings to chunks
    for i, (chunk, embedding) in enumerate(zip(batch_chunks, batch_embeddings)):
        all_embeddings_data.append({
            **chunk,
            "dense_embedding": embedding,
            "embedding_model": "e5-large",
            "embedding_dims": len(embedding)
        })
    
    progress = (batch_end / len(all_policy_chunks)) * 100
    print(f"   Progress: {batch_end}/{len(all_policy_chunks)} chunks ({progress:.1f}%)")

print(f"\n✅ Generated {len(all_embeddings_data)} E5-Large embeddings")
print(f"📊 Embedding dimension: {len(all_embeddings_data[0]['dense_embedding'])}")
print(f"📊 All embeddings normalized (L2)")

# Save embeddings locally (backup)
embeddings_backup = []
for data in all_embeddings_data:
    embeddings_backup.append({
        "chunk_id": data['chunk_id'],
        "embedding_dims": data['embedding_dims'],
        "embedding_model": data['embedding_model']
    })

with open("data/embeddings_metadata.json", 'w', encoding='utf-8') as f:
    json.dump(embeddings_backup, f, indent=2)

print("✅ Embeddings metadata backup: data/embeddings_metadata.json")


No sentence-transformers model found with name D:/models/e5-large. Creating a new one with mean pooling.


🔄 Loading E5-Large embeddings (multilingual, local)...
✅ E5-Large test embedding: 1024 dimensions
   Model: multilingual-e5-large (local)
   Embedding type: Dense (cosine similarity)
   Normalization: L2 normalized

🚀 Generating E5-Large embeddings for all chunks...
   Progress: 32/834 chunks (3.8%)
   Progress: 64/834 chunks (7.7%)
   Progress: 96/834 chunks (11.5%)
   Progress: 128/834 chunks (15.3%)
   Progress: 160/834 chunks (19.2%)
   Progress: 192/834 chunks (23.0%)
   Progress: 224/834 chunks (26.9%)
   Progress: 256/834 chunks (30.7%)
   Progress: 288/834 chunks (34.5%)
   Progress: 320/834 chunks (38.4%)
   Progress: 352/834 chunks (42.2%)
   Progress: 384/834 chunks (46.0%)
   Progress: 416/834 chunks (49.9%)
   Progress: 448/834 chunks (53.7%)
   Progress: 480/834 chunks (57.6%)
   Progress: 512/834 chunks (61.4%)
   Progress: 544/834 chunks (65.2%)
   Progress: 576/834 chunks (69.1%)
   Progress: 608/834 chunks (72.9%)
   Progress: 640/834 chunks (76.7%)
   Progress: 672/8

In [ ]:
# Build Pinecone-Ready Vectors (E5-Large) - format -> id(chunk id), values(1024 floats), metadata
# Means merge embedding and metadata
print("🔄 Building Pinecone vectors from E5 embeddings...")

pinecone_vectors = []

for data in all_embeddings_data:
    # Build metadata dict (banking + temporal critical)
    metadata = {
        **data['metadata'],  # All original metadata
        "chunk_id": data['chunk_id'],
        "chunk_index": data['chunk_index'],
        "text_length": data['text_length'],
        "word_count": data['word_count'],
        "embedding_model": "e5-large",
        "text_preview": data['text'][:200] + "..." if len(data['text']) > 200 else data['text']
    }
    
    # Pinecone vector (E5 dense only)
    vector = {
        "id": data['chunk_id'],
        "values": data['dense_embedding'],  # E5 dense (1024-dim)
        "metadata": metadata
    }
    
    pinecone_vectors.append(vector)

print(f"✅ Built {len(pinecone_vectors)} Pinecone vectors")
print(f"📏 Vector dimension: {len(pinecone_vectors[0]['values'])}")
print(f"📊 Metadata keys: {list(pinecone_vectors[0]['metadata'].keys())[:5]}...")

# Sample vector structure
print(f"\n🧪 Sample vector:")
print(f"   ID: {pinecone_vectors[0]['id']}")
print(f"   Dimension: {len(pinecone_vectors[0]['values'])}")
print(f"   Policy: {pinecone_vectors[0]['metadata']['policy_id']}")
print(f"   Status: {pinecone_vectors[0]['metadata'].get('status', 'N/A')}")

# Save Pinecone vectors to disk (backup before upload)
vectors_backup = []
for v in pinecone_vectors:
    vectors_backup.append({
        "id": v['id'],
        "dimension": len(v['values']),
        "metadata_keys": list(v['metadata'].keys())
    })

with open("data/pinecone_vectors_structure.json", 'w') as f:
    json.dump(vectors_backup, f, indent=2)

print(f"\n✅ Vector structure backup: data/pinecone_vectors_structure.json")
print(f"✅ Ready for Pinecone upload")


🔄 Building Pinecone vectors from E5 embeddings...
✅ Built 834 Pinecone vectors
📏 Vector dimension: 1024
📊 Metadata keys: ['policy_id', 'version_number', 'status', 'effective_date_ts', 'expiry_date_ts']...

🧪 Sample vector:
   ID: attached_onappointmentofstatutoryauditors_1_0000
   Dimension: 1024
   Policy: attached_onappointmentofstatutoryauditors
   Status: Active

✅ Vector structure backup: data/pinecone_vectors_structure.json
✅ Ready for Pinecone upload


In [12]:
# Initialize Pinecone Index (E5-Large config)
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Pinecone config 
PINECONE_INDEX_NAME = "fino-policies-e5"  # E5-specific index
E5_EMBEDDING_DIM = 1024  # E5-Large dimension

# Initialize Pinecone
print(f"🔄 Initializing Pinecone with API key...")
pc = Pinecone(api_key=PINECONE_API_KEY)

# List existing indexes
print(f"📋 Existing indexes: {[idx.name for idx in pc.list_indexes()]}")

# Check if index exists
if PINECONE_INDEX_NAME not in [idx.name for idx in pc.list_indexes()]:
    print(f"🔄 Creating Pinecone index: {PINECONE_INDEX_NAME}")
    print(f"   Dimension: {E5_EMBEDDING_DIM}")
    print(f"   Metric: cosine")
    print(f"   Type: Serverless")
    
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=E5_EMBEDDING_DIM,
        metric="cosine",  # E5 uses cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"✅ Index created: {PINECONE_INDEX_NAME}")
else:
    print(f"✅ Index exists: {PINECONE_INDEX_NAME}")

# Connect to index
index = pc.Index(PINECONE_INDEX_NAME)

# Get index stats
stats = index.describe_index_stats()
print(f"\n📊 Index stats:")
print(f"   Total vectors: {stats['total_vector_count']:,}")
print(f"   Namespaces: {list(stats['namespaces'].keys())}")
print(f"   Ready for vectors")


🔄 Initializing Pinecone with API key...
📋 Existing indexes: []
🔄 Creating Pinecone index: fino-policies-e5
   Dimension: 1024
   Metric: cosine
   Type: Serverless
✅ Index created: fino-policies-e5

📊 Index stats:
   Total vectors: 0
   Namespaces: []
   Ready for vectors


In [13]:
# Test E5 vector upsert (safety check)
print("🧪 Testing single E5 vector upsert...")

test_vector = pinecone_vectors[0]

# Test upsert to isolated namespace
test_upsert = index.upsert(
    vectors=[test_vector],
    namespace="fino-test-e5"
)

print(f"✅ Single upsert successful")
print(f"   Upserted ID: {test_vector['id']}")
print(f"   Response: {test_upsert}")

# Test retrieval (critical!)
print(f"\n🔍 Testing E5 retrieval...")

# Create test query embedding
test_query_text = "savings account minimum balance"
test_query_embedding = e5_embeddings.embed_query(test_query_text)

# Query test namespace
test_results = index.query(
    vector=test_query_embedding,
    top_k=3,
    namespace="fino-test-e5",
    include_metadata=True
)

print(f"✅ Query returned {len(test_results['matches'])} matches:")
for i, match in enumerate(test_results['matches'], 1):
    print(f"\n   [{i}] Score: {match['score']:.4f}")
    print(f"       ID: {match['id']}")
    print(f"       Policy: {match['metadata'].get('policy_id', 'N/A')}")
    print(f"       Preview: {match['metadata'].get('text_preview', 'N/A')[:100]}...")

print(f"\n✅ E5 vector upsert & retrieval working perfectly!")


🧪 Testing single E5 vector upsert...
✅ Single upsert successful
   Upserted ID: attached_onappointmentofstatutoryauditors_1_0000
   Response: UpsertResponse(upserted_count=1, _response_info={'raw_headers': {'date': 'Tue, 30 Dec 2025 06:40:10 GMT', 'content-type': 'application/json', 'content-length': '19', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '1', 'x-pinecone-request-logical-size': '5051', 'x-pinecone-request-latency-ms': '865', 'x-pinecone-request-id': '5484493862748635679', 'x-envoy-upstream-service-time': '380', 'x-pinecone-response-duration-ms': '867', 'grpc-status': '0', 'server': 'envoy'}})

🔍 Testing E5 retrieval...
✅ Query returned 1 matches:

   [1] Score: 0.7847
       ID: attached_onappointmentofstatutoryauditors_1_0000
       Policy: attached_onappointmentofstatutoryauditors
       Preview: POLICY FOR A


APPOINTMENT OF STA


AUDITORS TUTORY POLICY FOR A
 
APPOINTMENT OF STA
AUDITORS 
TUTO...

✅ E5 vector upsert & retrieval working perfectly!


In [28]:
# Full production batch upsert (E5)
print("🚀 PRODUCTION UPSERT: All E5 vectors to Pinecone")
print(f"📤 {len(pinecone_vectors):,} vectors → fino-policies namespace")

# Pinecone batch limits: 100MB / request, typically 1000 vectors safe
batch_size = 500
namespace = "fino-policies-e5"

total_upserted = 0
for batch_idx in range(0, len(pinecone_vectors), batch_size):
    batch_end = min(batch_idx + batch_size, len(pinecone_vectors))
    batch = pinecone_vectors[batch_idx:batch_end]
    
    # Upsert batch
    response = index.upsert(
        vectors=batch,
        namespace=namespace
    )
    
    total_upserted += len(batch)
    progress = (batch_end / len(pinecone_vectors)) * 100
    
    print(f"   Batch {(batch_idx // batch_size) + 1}: {len(batch)} vectors "
          f"({progress:.1f}%) → {response}")

print(f"\n✅ UPSERT COMPLETE!")
print(f"   Total vectors: {total_upserted:,}")
print(f"   Namespace: {namespace}")
print(f"   Model: E5-Large")

# Final index stats
final_stats = index.describe_index_stats()
print(f"\n📊 Final Pinecone stats:")
print(f"   Total vectors: {final_stats['total_vector_count']:,}")
print(f"   Namespaces: {list(final_stats['namespaces'].keys())}")
for ns, ns_stats in final_stats['namespaces'].items():
    print(f"      {ns}: {ns_stats['vector_count']:,} vectors")


🚀 PRODUCTION UPSERT: All E5 vectors to Pinecone
📤 834 vectors → fino-policies namespace
   Batch 1: 500 vectors (60.0%) → UpsertResponse(upserted_count=500, _response_info={'raw_headers': {'date': 'Tue, 30 Dec 2025 09:03:23 GMT', 'content-type': 'application/json', 'content-length': '21', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '1', 'x-pinecone-request-logical-size': '2472918', 'x-pinecone-request-latency-ms': '22440', 'x-pinecone-request-id': '7431179113503089301', 'x-envoy-upstream-service-time': '466', 'x-pinecone-response-duration-ms': '22462', 'grpc-status': '0', 'server': 'envoy'}})
   Batch 2: 334 vectors (100.0%) → UpsertResponse(upserted_count=334, _response_info={'raw_headers': {'date': 'Tue, 30 Dec 2025 09:03:50 GMT', 'content-type': 'application/json', 'content-length': '21', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '2', 'x-pinecone-request-logical-size': '1649101', 'x-pinecone-request-latency-ms': '15773', 'x-pinecone-request-id': '73998535671414